## Vamos a echar un ojo a todos los experimentos entrenados y coger el mejor

In [ ]:
import os
import mlflow
from mlflow.tracking import MlflowClient
from sklearn import datasets
import numpy as np

In [ ]:
# creamos una instancia del mlflow client
client = MlflowClient()

In [ ]:
#Obtenemos el experimento con el id
experiment = mlflow.get_experiment(experiment_id="<experiment-id>")

In [ ]:
# Obtenemos todas las ejecuciones del experimento
runs = client.search_runs(experiment_ids = [experiment.experiment_id])

In [ ]:
#Elegimos una metrica para elegir el mejor entrenamiento
metrics = [run.data.metrics["mse"] for run in runs]
best_rmse = min(metrics)

best_run = [run for run in runs if run.data.metrics["mse"] == best_rmse][0]
best_run.info

In [ ]:
# Registramos el modelo del mejor entrenamiento
model_name = "my best model"

model_uri = f"runs:/{best_run.info.run_id}/model"
model_version = mlflow.register_model(model_uri = model_uri, name = model_name)

## Cargamos el modelo

In [ ]:
model_info = client.get_registered_model(name = model_name)

In [ ]:
model_info

In [ ]:
model = mlflow.sklearn.load_model(model_uri)

## Hacemos inferencia

In [ ]:
diabetes = datasets.load_diabetes()
data = diabetes.data
preds = model.predict(data)

## Podemos añadir metadatos al modelo

In [ ]:
client.set_model_version_tag(model_name, model_version.version, key = "prediction_works", value = "true")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)

In [ ]:
model_version_info.tags

## Pasamos nuestro modelo registrado a "staging"

¡Que contentos con nuestro modelo! vamos a pasarlo a 'staging'

In [ ]:
client.transition_model_version_stage(model_name, model_version.version, stage = "Staging")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)

In [ ]:
# vemos si esta en staging
model_version_info.current_stage